In [1]:
import clickhouse_connect
import datetime
import os
import pytz
import pandas as pd
from dateutil.relativedelta import relativedelta
from dotenv import load_dotenv

import sys
sys.path.append('/home/boris/Documents/Work/analytics/Clickhouse')
from clickhouse_client import ClickHouse_client
ch = ClickHouse_client()
pd.set_option('display.max_rows', 1000)

___
## Tags: #Tables

# Links:

[[rep_mobile_citizens_id_city_partner]]

In [4]:
query_text = """--sql
    CREATE TABLE db1.t_activated_citizen_id
    (
        `report_date` Date,
        `address_uuid` String,
        `activated_citizen_id` UInt64,
        `flat_uuid` UInt64
    )
    ENGINE = MergeTree()
    ORDER BY address_uuid
    """
ch.query_run(query_text)

shape: (0, 0)
┌┐
╞╡
└┘

In [ ]:
start_date = datetime.datetime.strptime('2023-09-01','%Y-%m-%d').date()
end_date = datetime.datetime.strptime('2025-09-05','%Y-%m-%d').date()

dates_pd = pd.DataFrame({
        'date': pd.date_range(start=start_date, end=end_date).strftime('%Y-%m-%d'),
        'date_key': pd.date_range(start=start_date, end=end_date).strftime('%/year=%-Y/month=%m/%d%')
        })
dates_pd = dates_pd.iloc[::-1].reset_index(drop=True)
for day_index in dates_pd.index:
    date = dates_pd.loc[day_index,['date']].values[0]
    query_text = f"""
        INSERT INTO db1.t_activated_citizen_id
        SELECT
            report_date,
            address_uuid,
            countDistinctIf(citizen_id, citizen_id != 0) AS activated_citizen_id,	
            countDistinctIf(flat_uuid, flat_uuid != '') AS flat_uuid
        FROM db1.rep_mobile_citizens_id_city_partner
        WHERE address_uuid !=''
            AND report_date = '{date}'
        GROUP BY report_date,
                address_uuid
    """
    ch.query_run(query_text)
    print(date)

2025-09-05
2025-09-04
2025-09-03
2025-09-02
2025-09-01
2025-08-31
2025-08-30
2025-08-29
2025-08-28
2025-08-27
2025-08-26
2025-08-25
2025-08-24
2025-08-23
2025-08-22
2025-08-21
2025-08-20
2025-08-19
2025-08-18
2025-08-17
2025-08-16
2025-08-15
2025-08-14
2025-08-13
2025-08-12
2025-08-11
2025-08-10
2025-08-09
2025-08-08
2025-08-07
2025-08-06
2025-08-05
2025-08-04
2025-08-03
2025-08-02


In [ ]:
query_text = """--sql
    CREATE MATERIALIZED VIEW db1.t_activated_citizen_id_mv
    REFRESH EVERY 1 DAY OFFSET 5 HOUR 35 MINUTE TO db1.t_activated_citizen_id AS 
    SELECT
        report_date,
        address_uuid,
        countDistinctIf(citizen_id, citizen_id != 0) AS activated_citizen_id,	
        countDistinctIf(flat_uuid, flat_uuid != '') AS flat_uuid
    FROM db1.rep_mobile_citizens_id_city_partner
    WHERE address_uuid != ''
    GROUP BY report_date, address_uuid
    """
ch.query_run(query_text)

In [5]:
query_text = """--sql
SELECT
    *
FROM db1.t_activated_citizen_id
ORDER BY report_date DESC
LIMIT 10
    """
ch.query_run(query_text)

report_date,address_uuid,activated_citizen_id,flat_uuid
date,str,i64,i64
2025-09-06,"""ab2551ea-0514-46a0-8930-e316f1…",14,13
2025-09-06,"""ab266932-a642-4f6c-a331-a45cfa…",4,3
2025-09-06,"""ab2897b9-8f00-4a5d-8c92-463e3d…",10,4
2025-09-06,"""ab2c746c-c289-4612-9111-dedea5…",28,17
2025-09-06,"""ab2fdedf-3ddf-4033-b093-7dcaf5…",16,10
2025-09-06,"""ab306f26-8933-4a0c-b028-26ac59…",33,20
2025-09-06,"""ab4a5451-807a-40ef-bdb0-314615…",24,14
2025-09-06,"""ab31cc49-c684-440c-9320-6c7fb3…",36,20
2025-09-06,"""ab30b3db-9bc2-4160-b7e5-9d0149…",39,18


In [2]:
query_text = """
    DROP TABLE db1.t_activated_citizen_id
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,18537416,"""1f844653-5c80-4ab7-83e5-d723b5…"


In [3]:
query_text = """
    DROP TABLE db1.t_activated_citizen_id_mv
    """
ch.query_run(query_text)

read_rows,read_bytes,written_rows,written_bytes,total_rows_to_read,result_rows,result_bytes,elapsed_ns,query_id
i64,i64,i64,i64,i64,i64,i64,i64,str
0,0,0,0,0,0,0,2523682,"""5346be78-67b1-4600-82e3-d62ba8…"
